In [3]:
import sys

sys.path.append("..")
from icecube.dataset import IceCubeCasheDatasetV0
from icecube.utils import collate_fn
from icecube.utils import fit
from pathlib import Path
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch
import torch.nn.functional as F
from transformers.optimization import (
    get_linear_schedule_with_warmup,
    get_cosine_schedule_with_warmup,
)
from torch import nn
from x_transformers import ContinuousTransformerWrapper, Encoder, Decoder
from datasets import load_dataset, load_from_disk, concatenate_datasets
import pandas as pd
import numpy as np
import random

In [2]:
def angular_dist_score(
    az_true: torch.Tensor,
    zen_true: torch.Tensor,
    az_pred: torch.Tensor,
    zen_pred: torch.Tensor,
) -> torch.Tensor:
    sa1 = torch.sin(az_true)
    ca1 = torch.cos(az_true)
    sz1 = torch.sin(zen_true)
    cz1 = torch.cos(zen_true)

    sa2 = torch.sin(az_pred)
    ca2 = torch.cos(az_pred)
    sz2 = torch.sin(zen_pred)
    cz2 = torch.cos(zen_pred)

    scalar_prod = sz1 * sz2 * (ca1 * ca2 + sa1 * sa2) + cz1 * cz2
    scalar_prod = torch.clamp(scalar_prod, -1, 1)
    return torch.mean(torch.abs(torch.acos(scalar_prod)))





In [3]:
class CFG:
    DATA_CACHE_DIR = Path("../data/cache")
    BATCH_SIZE = 1024 * 2
    NUM_WORKERS = 16
    PRESISTENT_WORKERS = True
    LR = 1e-3
    WD = 1e-5
    WARM_UP_PCT = 0.1
    EPOCHS = 3
    FOLDER = 'EXP_05_HF'
    EXP_NAME = 'FIRST_EXP'



In [4]:
class LogCoshLoss(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, y_t, y_prime_t):
        ey_t = y_t - y_prime_t
        return torch.mean(torch.log(torch.cosh(ey_t + 1e-12)))



class MeanPoolingWithMask(nn.Module):
    def __init__(self):
        super(MeanPoolingWithMask, self).__init__()

    def forward(self, x, mask):
        # Multiply the mask with the input tensor to zero out the padded values
        x = x * mask.unsqueeze(-1)

        # Sum the values along the sequence dimension
        x = torch.sum(x, dim=1)

        # Divide the sum by the number of non-padded values (i.e. the sum of the mask)
        x = x / torch.sum(mask, dim=1, keepdim=True)

        return x

class FeedForward(nn.Module):
    def __init__(self, dim, dim_out = None, mult = 4):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim, dim * mult),
            nn.GELU(),
            nn.Linear(dim * mult, dim_out)
        )

    def forward(self, x):
        return self.net(x)


class IceCubeModelEncoderV0(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = ContinuousTransformerWrapper(
            dim_in=6,
            dim_out=128,
            max_seq_len=150,
            attn_layers=Encoder(dim=128,
                        depth=6, 
                        heads=8),
        )

        #self.pool = MeanPoolingWithMask()
        self.head = FeedForward(128, 2)

    def forward(self, x, mask):
        x = self.encoder(x, mask = mask)
        x = x.mean(dim=1)
        x = self.head(x)
        return x

#calculte metric based on angular distance
def get_score(y_hat, y):
    return angular_dist_score(y[:, 0], y[:, 1], y_hat[:, 0], y_hat[:, 1]).detach().cpu().numpy()

In [5]:
from copy import deepcopy
class HuggingFaceDatasetV0(torch.utils.data.Dataset):
    def __init__(self, ds, max_events=100):
        self.ds = ds
        self.max_events = max_events

    def __len__(self):
        return len(self.ds)

    def __getitem__(self, idx):
        item = self.ds[idx]

        event = pd.DataFrame(item)[
            [
                "time",
                "charge",
                "auxiliary",
                "x",
                "y",
                "z",
            ]
        ].astype(np.float32)
        if self.max_events:
            event = event[: self.max_events]
        event["time"] /= event["time"].max()
        event[["x", "y", "z"]] /= 500
        event["charge"] = np.log10(event["charge"])

        event = event.values
        mask = np.ones(len(event), dtype=bool)
        label = np.array([item["azimuth"], item["zenith"]], dtype=np.float32)

        batch = deepcopy(
            {
                "event": torch.tensor(event),
                "mask": torch.tensor(mask),
                "label": torch.tensor(label),
            }
        )
        return batch


def collate_fn(batch):
    event = [x["event"] for x in batch]
    mask = [x["mask"] for x in batch]
    label = [x["label"] for x in batch]

    event = torch.nn.utils.rnn.pad_sequence(event, batch_first=True)
    mask = torch.nn.utils.rnn.pad_sequence(mask, batch_first=True)
    batch = {"event": event, "mask": mask, "label": torch.stack(label)}
    return batch

In [6]:
PATH_DATAST = Path("../data")
nums = [i for i in range(1, 500)]
random.shuffle(nums)
trn_pth = [load_from_disk(PATH_DATAST / "hf_cashe" / f"batch_{i}.parquet") for i in nums]
trn_pth = concatenate_datasets(trn_pth)

vld_pth = [load_from_disk(PATH_DATAST / "hf_cashe" / f"batch_{i}.parquet") for i in range(600, 603)]
vld_pth = concatenate_datasets(vld_pth)


print(len(trn_pth), len(vld_pth))

trn_ds = HuggingFaceDataset(trn_pth)
vld_ds = HuggingFaceDataset(vld_pth)

trn_dl = DataLoader(
    trn_ds,
    batch_size=CFG.BATCH_SIZE,
    shuffle=False,
    num_workers=CFG.NUM_WORKERS,
    persistent_workers=CFG.PRESISTENT_WORKERS,
    drop_last=True,
    collate_fn=collate_fn,
)
vld_dl = DataLoader(
    vld_ds,
    batch_size=CFG.BATCH_SIZE,
    shuffle=False,
    num_workers=CFG.NUM_WORKERS,
    persistent_workers=CFG.PRESISTENT_WORKERS,
    drop_last=False,
    collate_fn=collate_fn,
)


custom_model = IceCubeModelEncoderV0()
opt = torch.optim.AdamW(custom_model.parameters(), lr=CFG.LR, weight_decay=CFG.WD)
loss_func = LogCoshLoss()
warmup_steps = int(len(trn_dl) * int(CFG.WARM_UP_PCT * CFG.EPOCHS))
total_steps = int(len(trn_dl) * CFG.EPOCHS)
sched = get_linear_schedule_with_warmup(
    opt, num_warmup_steps=warmup_steps, num_training_steps=total_steps
)

fit(
    epochs=CFG.EPOCHS,
    model=custom_model,
    train_dl=trn_dl,
    valid_dl=vld_dl,
    loss_fn=loss_func,
    opt=opt,
    metric=get_score,
    folder=CFG.FOLDER,
    exp_name=f"{CFG.EXP_NAME}",
    device="cuda:0",
    sched=sched,
)

/usr/local/lib/python3.8/dist-packages/datasets/arrow_dataset.py:1533: FutureWarning: 'fs' was is deprecated in favor of 'storage_options' in version 2.8.0 and will be removed in 3.0.0.
You can remove this warning by passing 'storage_options=fs.storage_options' instead.
  warnings.warn(


79800000 600000


epoch,train_loss,valid_loss,val_metric


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceed

Better model found at epoch 1 with value: 1.2050968408584595.
   epoch  train_loss  valid_loss     metric
0      1    0.487884    0.483495  1.2050968


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceed

In [5]:
PATH_DATAST = Path("../data")
nums = [i for i in range(1, 3)]
random.shuffle(nums)
trn_pth = [load_from_disk(PATH_DATAST / "hf_cashe" / f"batch_{i}.parquet") for i in nums]
trn_pth = concatenate_datasets(trn_pth)

In [9]:
torch.get_num_threads()

24

In [11]:
torch.get_num_interop_threads()

24

In [12]:
torch.cuda.device_count()

2

In [13]:
import os

In [14]:
os.cpu_count()

48

In [17]:
from datasets import Dataset
import pyarrow.parquet as pq

In [23]:
ds=Dataset(pq.read_table("../data/train/batch_1.parquet", memory_map=True))

In [25]:
ds[0:1000]

{'sensor_id': [3918,
  4157,
  3520,
  5041,
  2948,
  860,
  2440,
  1743,
  3609,
  5057,
  5057,
  2977,
  5059,
  3496,
  3161,
  2959,
  1397,
  1970,
  3387,
  1583,
  1940,
  1241,
  558,
  557,
  1405,
  557,
  558,
  557,
  557,
  3050,
  553,
  972,
  973,
  2261,
  975,
  560,
  554,
  3276,
  4831,
  4571,
  3520,
  3699,
  300,
  613,
  3438,
  2421,
  3609,
  3115,
  5057,
  4528,
  3496,
  2448,
  3289,
  3050,
  4904,
  1970,
  3452,
  48,
  3267,
  3267,
  104,
  458,
  4107,
  502,
  2824,
  607,
  1938,
  763,
  4811,
  1911,
  4502,
  4503,
  4506,
  4892,
  4507,
  4502,
  4148,
  4083,
  3668,
  3674,
  4594,
  3676,
  4440,
  3140,
  3139,
  3140,
  3140,
  3141,
  3840,
  3142,
  1836,
  5067,
  2142,
  3832,
  2152,
  2214,
  492,
  919,
  1151,
  1028,
  4312,
  752,
  2681,
  3361,
  4107,
  4107,
  4739,
  690,
  171,
  4503,
  4065,
  1287,
  4685,
  3925,
  15,
  2583,
  413,
  869,
  2616,
  3779,
  3778,
  3778,
  3778,
  3778,
  3777,
  3778,
  3777,
  